In [ ]:
import os
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import MaxPooling2D
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Lambda
import nbimporter
from siamese import create_pairs, process_images

data_directory = 'sketch_small'
TARGET_SIZE = (64, 64, 1)
EPOCHS = 50
BATCH_SIZE = 32

In [ ]:
def contrastiveLoss(y, y_preds, margin=1):
    y = tf.cast(y, y_preds.dtype)
    y_preds_squared = K.square(y_preds)
    margin_squared = K.square(K.maximum(margin - y_preds, 0))
    loss = K.mean(y * y_preds_squared + (1 - y) * margin_squared)
    return loss

In [ ]:
def euclidean_distance(vecs):
    (imgA, imgB) = vecs
    ss = K.sum(K.square(imgA - imgB), axis = 1, keepdims=True)
    return K.sqrt(K.maximum(ss, K.epsilon()))

In [ ]:
def siamese_model(input_shape, embeddingDim = 48):
    inputs = Input(input_shape)
    x = Conv2D(128, (2, 2), padding = "same", activation = "relu")(inputs)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.3)(x)

    x = Conv2D(128, (2, 2), padding = "same", activation = "relu")(inputs)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.3)(x)


    pooling = GlobalAveragePooling2D()(x)
    outputs = Dense(embeddingDim)(pooling)
    model = Model(inputs, outputs)


    return model

In [ ]:
X, y = create_pairs(data_directory)
X = process_images(X, TARGET_SIZE)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train, X_test, y_train, y_test = np.array(X_train), np.array(X_test), np.array(y_train), np.array(y_test)


In [ ]:
X_train, X_test = X_train / 255.0, X_test / 255.0

X_train = np.expand_dims(X_train, axis = -1)
X_test = np.expand_dims(X_test, axis = -1)
y_train = np.expand_dims(y_train, axis = -1)
y_test = np.expand_dims(y_test, axis = -1)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
imageA = Input(shape = TARGET_SIZE)
imageB = Input(shape = TARGET_SIZE)

model_build = siamese_model(TARGET_SIZE)
modelA = model_build(imageA)
modelB = model_build(imageB)

distance = Lambda(euclidean_distance)([modelA, modelB])
model = Model(inputs=[imageA, imageB], outputs=distance)



In [ ]:
X_train_left = X_train[:, 0]
X_train_right = X_train[:, 1]
X_test_left = X_test[:, 0]
X_test_right = X_test[:, 1]

In [ ]:
model.compile(loss = contrastiveLoss, optimizer="adam")
history = model.fit(
    [X_train_left, X_train_right], y_train[:],
    validation_data=([X_test_left, X_test_right], y_test[:]),
    batch_size = BATCH_SIZE,
    epochs = EPOCHS)